In [1]:
# Download Data
!git clone https://github.com/deeplearningzerotoall/PyTorch.git
!mv PyTorch/custom_data/origin_data/ ./
!mv PyTorch/custom_data/test_data/ ./

Cloning into 'PyTorch'...
remote: Enumerating objects: 1899, done.
remote: Total 1899 (delta 0), reused 0 (delta 0), pack-reused 1899
Receiving objects: 100% (1899/1899), 80.33 MiB | 44.04 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [2]:
import torch
from torch import nn

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

import random
from matplotlib.pyplot import imshow
import os

# check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# set seed for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

torch.__version__, device

('1.7.0+cu101', 'cuda')

## ImageFolder
- 원본 사진이 256x512 사이즈의 고해상도 사진이므로, 용량을 낮추기 위해 불러올 때 64x128 사이즈로 변환
- torchvision.transforms.Compose() 를 사용하면 여러 단계의 transform을 묶어 한 번에 적용 가능


In [3]:
# Load train dataset
trans = transforms.Compose([
    transforms.Resize((64, 128)),
    transforms.ToTensor(),
])

train_data = torchvision.datasets.ImageFolder(root='origin_data/', transform=trans)
data_loader = DataLoader(train_data, batch_size=10, shuffle=True)

train_data[0][0].shape, data_loader

(torch.Size([3, 64, 128]),
 <torch.utils.data.dataloader.DataLoader at 0x7f4ce86b2c50>)

## Save & Load model (state_dict)
- `torch.save(model.state_dict(), 'model.pth')` 명령어를 사용해 학습된 모델 가중치(파라미터) 저장 가능
- 저장된 모델과 동일한 아키텍쳐로 만들어진 모델 객체를 초기화한 후, `new_model.load_state_dict(torch.load('model.pth'))` 명령어를 사용해 기존에 학습했던 모델 가중치(파라미터) 재사용 가능

In [4]:
# Generate layers
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(6, 16, 5, 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
    )
    self.layer3 = nn.Sequential(
        nn.Linear(in_features=16*13*29, out_features=120, bias=True),
        nn.ReLU(),
        nn.Linear(120, 2)
    )
  
  def forward(self, x, verbose=False):
    def report_shape(output, verbose=verbose):
      if verbose:
        print(output.shape)

    output = self.layer1(x)
    report_shape(output)
    
    output = self.layer2(output)
    report_shape(output)
    
    output = output.view(output.shape[0], -1)
    report_shape(output)
    
    output = self.layer3(output)
    report_shape(output)
    
    return output

In [5]:
# Test layer outputs - checking output shape of each layer
net = CNN().to(device)
test_in = torch.Tensor(size=(10, 3, 64, 128)).to(device)
test_out = net.forward(test_in, verbose=True)

torch.Size([10, 6, 30, 62])
torch.Size([10, 16, 13, 29])
torch.Size([10, 6032])
torch.Size([10, 2])


In [6]:
%%time

# Train model
optimizer = torch.optim.Adam(net.parameters(), lr=0.00005)
criterion = nn.CrossEntropyLoss().to(device)

n_epochs = 5
num_batch = len(data_loader)

for epoch in range(n_epochs):
  avg_cost = 0
  for idx, data in enumerate(data_loader):
    value, label = data[0].to(device), data[1].to(device)

    hypothesis = net.forward(value)
    cost = criterion(hypothesis, label)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost/num_batch
  print(f'epoch : {epoch+1:3}  |  cost : {avg_cost:10.6f}')
print('>>> Train Finished!')

# Save trained weights (state_dict)
os.makedirs('model', exist_ok=True)
torch.save(net.state_dict(), 'model/model.pth')

epoch :   1  |  cost :   0.639913
epoch :   2  |  cost :   0.462027
epoch :   3  |  cost :   0.195709
epoch :   4  |  cost :   0.064646
epoch :   5  |  cost :   0.027660
>>> Train Finished!
CPU times: user 7.92 s, sys: 136 ms, total: 8.06 s
Wall time: 8.12 s


In [7]:
# Load & Check model's state_dict
new_net = CNN().to(device)
new_net.load_state_dict(torch.load('model/model.pth'))
new_net.eval()
print()

print(net.layer1[0])
print(new_net.layer1[0], '\n')

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0], '\n')

(net.layer1[0].weight == new_net.layer1[0].weight).all().item()


Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1)) 

tensor([-0.0934,  0.0012, -0.0192, -0.0234,  0.0909], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([-0.0934,  0.0012, -0.0192, -0.0234,  0.0909], device='cuda:0',
       grad_fn=<SelectBackward>) 



True

## Test model performance

In [8]:
# Load test dataset
trans = transforms.Compose([
    transforms.Resize((64, 128)),
    transforms.ToTensor(),
])

test_data = torchvision.datasets.ImageFolder(root='test_data/', transform=trans)
data_loader_test = DataLoader(test_data, batch_size=len(test_data))

In [9]:
%%time

# Test model with test dataset
with torch.no_grad():
  for data in data_loader_test:
    value, label = data[0].to(device), data[1].to(device)
    
    prediction = new_net.forward(value).argmax(1)
    acc = (prediction == label).float().mean()
    print(f'>>> Test accuracy : {acc:6.4f}')

>>> Test accuracy : 1.0000
CPU times: user 2.53 s, sys: 21.2 ms, total: 2.56 s
Wall time: 2.56 s
